<center><span style="font-size:36px;">  🏈Raw Tackle Ability and Tackle Index🏈</span> </center>

***

### <font color='289C4E'  align>Table of contents<font><a class='anchor' id='top'></a>
- [Import Data](#1)
- [Standardize Directional Data](#2)
- [Merge the Data](#3)
- [Create summary of game](#4)
- [Exploratory Data Analysis](#5)
- [Modeling](#6)
- [Explaining the Model](#7)
- [Stats by Tackler](#8)
- [NFL Use Cases](#9)
- [GIF/Animation](#10) 


***

## Introduction

At a basic level, tackling is crucial to the game of football in several ways:
 - **Preventing yards gained:** Defensive tackling is a key way to prevent the offensive team from gaining additiongl yardage. When executed at the right moment, a successful tackle can put the opposing team in unfavorable conditions.
 - **Preventing scoring:** When every point counts, a successful tackle can make the difference between winning and losing a game. By preventing a player from reaching the end zone and forcing the opposing team to attempt field goals, tacklers limit points scored.
 - **Creating turnovers:** If a defender can tackle effectively and lead to a forced fumble, their own team can gain posession of the ball and get the chance to score more points.
 
 

### Project Goal:
Our goal is to analyze historical tracking data and extract meaningful features to predict when a tackle attempt will be successful. We will then apply the model to players and determine their "Raw Tackle Ability", which we define as "the probability that a player will make a successful tackle based on their previous behavior."

We also introduce the "Tackle Index", which allows us to rank players based on their current track record related to tackling in addition to their Raw Tackle Ability (RTA). This accounts for consistent performance and number of games played.

***

# 1. Import Data <a class="anchor"  id="1"></a>

The first thing I'm going to do is read all of the data. For the tracking data, I'm going to concatenate the dataframes so that there is one comprehensive dataframe with all of the tracking info.

In [1]:
import warnings
#!pip install eli5
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

import pandas as pd
import numpy as np
"""df_games=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/games.csv')
df_players=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/players.csv')
df_plays=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/plays.csv')
df_tackles=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/tackles.csv')
files=[]
for i in range(1,10):
    file='/kaggle/input/nfl-big-data-bowl-2024/tracking_week_'+str(i)+'.csv'
    files.append(pd.read_csv(file))
    df_tracking=pd.concat(files) 
"""


"df_games=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/games.csv')\ndf_players=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/players.csv')\ndf_plays=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/plays.csv')\ndf_tackles=pd.read_csv('/kaggle/input/nfl-big-data-bowl-2024/tackles.csv')\nfiles=[]\nfor i in range(1,10):\n    file='/kaggle/input/nfl-big-data-bowl-2024/tracking_week_'+str(i)+'.csv'\n    files.append(pd.read_csv(file))\n    df_tracking=pd.concat(files) \n"

In [2]:
df_games=pd.read_csv('games.csv')
df_players=pd.read_csv('players.csv')
df_plays=pd.read_csv('plays.csv')
df_tackles=pd.read_csv('tackles.csv')
files=[]
for i in range(1,10):
    file='tracking_week_'+str(i)+'.csv'
    files.append(pd.read_csv(file))
    df_tracking=pd.concat(files)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df_tackles.describe()

# 2. Standardize Directional Data <a class="anchor"  id="2"></a>

I used this notebook to standardize all of my directional data: 
https://www.kaggle.com/code/colinlagator/make-all-plays-left-right 

In [ ]:
#change all plays to same direction
def reverse_deg(deg):
    if deg < 180:
        return deg + 180
    if deg >= 180:
        return deg - 180


In [ ]:
df_tracking["o_standard"]=np.where(df_tracking["playDirection"] == "left", df_tracking["o"].apply(reverse_deg), df_tracking["o"])
        
df_tracking["dir_standard"] = np.where(df_tracking["playDirection"] == "left", df_tracking["dir"].apply(reverse_deg), df_tracking["dir"])
        
df_tracking["x_standard"] = np.where(df_tracking["playDirection"] == "left", df_tracking["x"].apply(lambda x: 120 - x), df_tracking["x"])
        
df_tracking["y_standard"] =np.where(df_tracking["playDirection"] == "left",  df_tracking["y"].apply(lambda y: 160/3 - y), df_tracking["y"])
    

In [ ]:
def create_gameplayid(df):
    df['gameplayid']=df['gameId'].astype(str)+df['playId'].astype(str)
    return df

def create_index(df):
    df['index']=df['nflId'].astype(str)+df['gameplayid']
    return df
df_tracking=create_index(create_gameplayid(df_tracking))

df_plays=create_gameplayid(df_plays)
df_tackles=create_index(create_gameplayid(df_tackles))



In [ ]:
print('Distinct plays in tracking data: ', df_tracking['gameplayid'].nunique())
print('Distinct plays in play data: ', df_plays['gameplayid'].nunique())
print('Distinct plays in tackles data: ', df_tackles['gameplayid'].nunique())

# 3. Merge the data <a class="anchor"  id="3"></a>

In [ ]:
def merge_clean_data(df_tracking, df_plays, df_tackles):
    df_tracking_plays_final=pd.merge(pd.merge(df_tracking, df_plays, left_on=['gameplayid', 'gameId', 'playId'], right_on=['gameplayid', 'gameId', 'playId'], how='inner'), df_tackles, left_on=['gameplayid', 'gameId', 'playId', 'nflId'], right_on=['gameplayid', 'gameId', 'playId', 'nflId'], how='inner' )
    df_tracking_plays_final['event']=np.where(df_tracking_plays_final['tackle']==1, 'tackle', np.where(df_tracking_plays_final['pff_missedTackle']==1, 'missed_tackle', (np.where(df_tracking_plays_final['assist']==1, 'assist', 'Other')) ))
    df_tackles_final=pd.merge(df_tracking_plays_final, df_tackles, left_on=['gameplayid','gameId', 'playId', 'nflId', 'tackle', 'assist', 'pff_missedTackle'], right_on=['gameplayid','gameId', 'playId', 'nflId', 'tackle', 'assist', 'pff_missedTackle'], how='inner', suffixes=['_1', '_2'])
    df_tackles_final=df_tackles_final[( (df_tackles_final['tackle']+df_tackles_final['assist']+df_tackles_final['pff_missedTackle']>=1)&(df_tackles_final['gameplayid'].isna()==False))]
    df_tackles_final['index']=df_tackles_final['nflId'].astype(str)+df_tackles_final['gameplayid'].astype(str)
    return df_tackles_final
df_tackles_final=merge_clean_data(df_tracking, df_plays, df_tackles)

In [ ]:

print('Distinct plays in  final data: ', df_tackles_final['gameplayid'].nunique())
print('Distinct frames in  final data: ', df_tackles_final['index'].nunique())

In [ ]:
def ballcarrier_df(df_players):
    df_ballcarrier_players=df_players.copy()
    df_ballcarrier_players.columns=['nflId_ballcarrier', 'ballCarrierHeight', 'ballCarrierWeight', 'ballcarrierDOB', 'ballcarrierCollege', 'ballcarrierPosition', 'ballCarrierDisplayName']
    
    df_tracking_ballcarrier=df_tracking[['gameplayid', 'frameId','nflId', 'x', 'y', 's', 'a', 'dis',
       'o', 'dir', 'event', 'o_standard', 'dir_standard', 'x_standard',
       'y_standard']]
    df_ballcarrier=pd.merge(df_tracking_ballcarrier, df_ballcarrier_players, left_on='nflId', right_on='nflId_ballcarrier')
    df_ballcarrier=df_ballcarrier[['gameplayid', 'frameId',  'x', 'y', 's', 'a', 'dis', 'o',
       'dir', 'event', 'o_standard', 'dir_standard', 'x_standard',
       'y_standard', 'nflId_ballcarrier', 'ballCarrierHeight',
       'ballCarrierWeight', 'ballcarrierDOB', 'ballcarrierCollege',
       'ballcarrierPosition', 'ballCarrierDisplayName']]
    df_ballcarrier.columns=['gameplayid', 'frameId',  'x_ballcarrier', 'y_ballcarrier', 's_ballcarrier',
                        'a_ballcarrier', 'dis_ballcarrier', 'o_ballcarrier',
       'dir_ball_carrier', 'event_ballcarrier', 'o_standard_ballcarrier', 'dir_standard_ballcarrier', 'x_standard_ballcarrier',
       'y_standard_ballcarrier','nflId_ballcarrier', 'ballCarrierHeight',
       'ballCarrierWeight', 'ballcarrierDOB', 'ballcarrierCollege',
       'ballcarrierPosition', 'ballCarrierDisplayName']
    return df_ballcarrier
    
df_ballcarrier=ballcarrier_df(df_players)
df_ballcarrier.head()

In [ ]:
def full_merge(df_tackles_final):
    df_players_tackler=df_players.copy()
    df_players_tackler.columns=['nflId_tackler', 'tacklerHeight', 'tacklerWeight', 'tacklerDOB', 'tacklerCollege', 'tacklerPosition', 'tacklerDisplayName']
    df_tackles_final=pd.merge(df_tackles_final, df_players_tackler, left_on='nflId', right_on='nflId_tackler')
    df_full=pd.merge(df_tackles_final, df_ballcarrier, left_on=['gameplayid','ballCarrierDisplayName','frameId'], right_on=['gameplayid', 'ballCarrierDisplayName','frameId'])
    df_full.rename(columns={'nflId':'nflId_tackler', 'x':'x_tackler','y': 'y_tackler', 's':'s_tackler', 'a':'a_tackler', 'dis':'dis_tackler',
       'o':'o_tackler', 'dir':'dir_tackler', 'event':'event_tackler', 'o_standard':'o_standard_tackler',
       'dir_standard':'dir_standard_tackler', 'x_standard':'x_standard_tackler', 'y_standard':'y_standard_tackler'}, inplace=True)
    return df_full
df_full=full_merge(df_tackles_final)

## Key Metrics from Physics
We decided to borrow a few concepts from basic physics to calculate additional metrics for the players (tackler and ball carrier). 
 - Force = Mass x Acceleration
 - Momentum = Mass x Speed
 - Relative velocity
 
 # Relative Velocity and the Coefficient of Restitution: 
- In physics, there are two types of collisions: elastic and inelastic. An elastic collision results in objects moving apart at the same rate at which they collided. On the other hand, an inelastic collision results in objects staying stuck together. According to [The University of Arkansas ](https://phys.libretexts.org/Courses/Merrimack_College/Conservation_Laws_Newton's_Laws_and_Kinematics_version_2.0/08%3A_C8_Conservation_of_Energy-_Kinetic_and_Gravitational/8.05%3A_Relative_Velocity_and_the_Coefficient_of_Restitution) you can quantify how elastic a collision is with the coefficient of restitution (COR). Typically football tackles are classified as inelastic collisions, but we'd like to use the COR to quantify how inelastic a tackle is.

We are suggesting an adjusted COR which adds 1 to the numerator and denominator to account for division by 0.
 

## COR= 
## $\frac{(v_{2f} - v_{1f})+1}{(v_{2i} - v_{1i})+1}$

In [ ]:
def create_force_features(df_full):
    df_full['force_ballcarrier']=df_full['ballCarrierWeight']*df_full['a_ballcarrier']
    df_full['force_tackler']=df_full['tacklerWeight']*df_full['a_tackler']
    df_full['momentum_ballcarrier']=df_full['ballCarrierWeight']*df_full['s_ballcarrier']
    df_full['momentum_tackler']=df_full['tacklerWeight']*df_full['s_tackler']
    #calculate distance between player tackling and player being tackled
    df_full['dis_ballcarrier_tackler']=np.sqrt(((df_full['x_standard_ballcarrier'] -df_full['x_standard_tackler'])**2)+((df_full['y_standard_ballcarrier'] -df_full['y_standard_tackler'])**2))
    return df_full
df_full=create_force_features(df_full)


In [ ]:
import time

def minutes_to_seconds(minutes):
  return minutes * 60

def seconds_to_minutes(seconds):
  return seconds // 60
def convert_minutes_to_seconds(hours_minutes_and_seconds):
  hours, minutes, seconds = hours_minutes_and_seconds.split(':')
  return minutes_to_seconds(int(minutes)) + seconds_to_minutes(float(seconds))




In [ ]:
import datetime 
from datetime import datetime

df_full['time']=df_full['time'].apply(lambda x: datetime.strptime(x,'%Y-%m-%d %H:%M:%S.%f' ))
df_full['time_adj']=df_full['time'].apply(lambda x: x.time())
df_full['time_seconds']=df_full['time_adj'].apply(lambda x: convert_minutes_to_seconds(str(x)))


In [ ]:

df_full['time_elapsed']=df_full.groupby(['gameplayid', 'frameId'])['time_seconds'].diff()
df_full['nflId_tackler']=df_full['nflId_tackler'].astype(str)
df_full['nflId_ballcarrier']=df_full['nflId_ballcarrier'].astype(str)


In [ ]:
df_full.sort_values(by=['gameplayid', 'frameId'], inplace=True)


In [ ]:

df_full['relative_direction']=df_full['dir_standard_tackler']-df_full['dir_standard_ballcarrier']
df_full['relative_o']=df_full['o_standard_tackler']-df_full['o_standard_ballcarrier']
df_full['playNullifiedByPenalty'].replace({'Y':1, 'N':0}, inplace=True)

In [ ]:
df_groupplay=df_full.groupby(by=['gameplayid', 'displayName'])['dis_ballcarrier_tackler'].min().reset_index()
df_groupplay['tackleframe']=1
df_full=pd.merge(df_groupplay, df_full, left_on=['gameplayid', 'displayName', 'dis_ballcarrier_tackler'], right_on=['gameplayid','displayName', 'dis_ballcarrier_tackler'], how='right')
df_full['tackle_frame_final']=np.where((df_full['tackleframe']==1), 1, 0)
df_full['prior_speed_tackler']=df_full.groupby(['gameplayid', 'displayName'])['s_tackler'].shift(1)
df_full['next_speed_tackler']=df_full.groupby(['gameplayid', 'displayName'])['s_tackler'].shift(-1)
df_full['prior_speed_ballcarrier']=df_full.groupby(['gameplayid', 'displayName'])['s_ballcarrier'].shift(1)
df_full['next_speed_ballcarrier']=df_full.groupby(['gameplayid', 'displayName'])['s_ballcarrier'].shift(-1)


In [ ]:
def group_by_play(df_full):
    df_full_grouped=df_full.groupby(by=['gameplayid',  'displayName']).agg({'playNullifiedByPenalty':'max', 'relative_o':'mean', 'relative_direction':'mean', 'time':['min', 'max'], 'time_seconds':['min', 'max'],'offenseFormation':pd.Series.mode,'tacklerPosition': pd.Series.mode, 'ballcarrierPosition':pd.Series.mode,'offenseFormation':'first',  'x_tackler':['min', 'max','mean'], 'y_tackler':['min', 'max','mean'],
                                  's_tackler':['first', 'last','min', 'max','mean'], 'a_tackler':['min', 'max','mean'], 'dis_tackler':['min', 'max','mean'],
                                  'o_tackler':['min', 'max','mean'], 'dir_tackler':['min', 'max','mean'], 'o_standard_tackler':['min', 'max','mean'],
                                  'dir_standard_tackler':['min', 'max','mean'], 'x_standard_tackler':['min', 'max','mean'], 'y_standard_tackler':['min', 'max','mean'],
                                  'quarter':['min', 'max','mean'], 'down':['min', 'max','mean'], 'yardsToGo':['min', 'max','mean'],
                                  'gameClock':['min', 'max'], 'preSnapHomeScore':['min', 'max','mean'], 'preSnapVisitorScore':['min', 'max','mean'],
                                  'passLength':['min', 'max','mean'], 'absoluteYardlineNumber':['min', 'max','mean'], 'defendersInTheBox': ['min', 'max','mean'],
                                  'expectedPoints':['min', 'max','mean'], 'tackle':'max', 'assist':'max',  'pff_missedTackle':'max', 'tacklerWeight': 'mean',
                                  'x_ballcarrier': ['min', 'max','mean'], 'y_ballcarrier':['min', 'max','mean'], 's_ballcarrier':['first', 'last','min', 'max','mean'],
                                  'a_ballcarrier':['min', 'max','mean'], 'dis_ballcarrier': ['min', 'max','mean'], 'o_ballcarrier':['min', 'max','mean'],
                                  'o_standard_ballcarrier':['min', 'max','mean'],'dir_standard_ballcarrier':['min', 'max','mean'],
                                  'x_standard_ballcarrier':['min', 'max','mean'], 'y_standard_ballcarrier':['min', 'max','mean'], 'ballCarrierWeight':'mean', 'force_ballcarrier':['min', 'max','mean'],
                                  'force_tackler':['min', 'max','mean']}).reset_index()
    df_full_grouped.columns =df_full_grouped.columns.map('|'.join).str.strip('|')
    return df_full_grouped
df_full_grouped=group_by_play(df_full)


## Summary Statistics

In [ ]:
print('Total plays: ', df_full_grouped['gameplayid'].nunique())
print('Total plays not nullified by penalty: ', df_full_grouped[df_full_grouped['playNullifiedByPenalty|max']==0]['gameplayid'].nunique())
print('Total row count: ', len(df_full_grouped[df_full_grouped['playNullifiedByPenalty|max']==0]))
print('Total missed tackles: ', len(df_full_grouped[(df_full_grouped['pff_missedTackle|max']==1)]['gameplayid']))
print('Total assists: ', len(df_full_grouped[((df_full_grouped['assist|max']==1)&(df_full_grouped['pff_missedTackle|max']==0))]['gameplayid']))
print('Total tackles: ', len(df_full_grouped[((df_full_grouped['tackle|max']==1)&(df_full_grouped['assist|max']==0)&(df_full_grouped['pff_missedTackle|max']==0))]['gameplayid']))

In [ ]:
##remove plays nullified by penalty 
df_full_grouped=df_full_grouped[df_full_grouped['playNullifiedByPenalty|max']==0]

In [ ]:
df_speed_subset=df_full[df_full['tackle_frame_final']==1][['gameplayid', 'displayName', 's_tackler', 'prior_speed_tackler', 'next_speed_tackler', 's_ballcarrier', 'prior_speed_ballcarrier', 'next_speed_ballcarrier']]

In [ ]:
df_speed_subset.fillna(0, inplace=True)

Just for an extra resource, I'm going to create descriptions of each game by adding all the play descriptions together for each game.

# 4. Create summary of game <a class="anchor"  id="4"></a>

In [ ]:
df_playdescription = df_plays[['gameId', 'playId', 'playDescription', 'gameClock']].sort_values(by=['gameId',  'gameClock']).set_index('gameId')
df_gamedescription=df_playdescription.groupby(['gameId'])[['playDescription']].transform(lambda x: ','.join(x)).reset_index().drop_duplicates()
df_gamedescription.rename(columns={'playDescription':'gameDescription'}, inplace=True)

In [ ]:
df_gamedescription.head()

# 5. Exploratory Data Analysis <a class="anchor"  id="5"></a>

Now that I have a final dataframe with one row per play, I'm going to explore the various features that can relate to a successful or unsuccessful tackle. Part of this involves merging more data related to the player who is tackling (tackler) and the ball carrier. 
 

In [ ]:
def create_position_variables(df_full_grouped):

    ballcarrier_position=pd.get_dummies(df_full_grouped[['gameplayid',  'displayName', 'ballcarrierPosition|mode']], columns=['ballcarrierPosition|mode'],drop_first=True).reset_index(drop=True)
    ballcarrier_position.columns=['gameplayid', 'displayName', 'QB_ballcarrier', 'RB_ballcarrier', 'TE_ballcarrier', 'WR_ballcarrier']
    tackler_position=pd.get_dummies(df_full_grouped[['gameplayid','displayName', 'tacklerPosition|mode']], columns=['tacklerPosition|mode'],drop_first=True).reset_index(drop=True)
    tackler_position.columns=['gameplayid', 'displayName', 'tacklerPosition|mode_DB', 'tacklerPosition|mode_DE',
       'tacklerPosition|mode_DT', 'tacklerPosition|mode_FS',
       'tacklerPosition|mode_ILB', 'tacklerPosition|mode_MLB',
       'tacklerPosition|mode_NT', 'tacklerPosition|mode_OLB',
       'tacklerPosition|mode_SS']
    offense_formation=pd.get_dummies(df_full_grouped[['gameplayid', 'displayName','offenseFormation|first']], columns=['offenseFormation|first'], drop_first=True).reset_index(drop=True)
    df_merge=pd.merge(df_full_grouped,ballcarrier_position, on=['gameplayid', 'displayName'] )
    df_merge=pd.merge(df_merge,tackler_position, on=['gameplayid', 'displayName'])
    df_merge=pd.merge(df_merge, offense_formation, on=['gameplayid', 'displayName'])
    
    
    return df_merge
df_full_grouped=create_position_variables(df_full_grouped)

### Target Variable
Our final target variable is the "tackle assist success". This is a binary field that flags if a tackle or assist occured during the play (1) or if a tackle was missed (0).
We are also going to try to use tackle success as a target. This means we will only count solo tackles and missed tackles. Assists will be disregarded.

Out of 17,456 events in the tracking data, 15,365 resulted in a "tackle assist success" and 2,091 were missed tackles. This brings our base success rate to 88%.

In [ ]:
#create final target variable
df_full_grouped['tackle_assist_success']=np.where(((df_full_grouped['tackle|max']+df_full_grouped['assist|max']>=1)& (df_full_grouped['pff_missedTackle|max']==0)), 1, 0)


In [ ]:
df_final=df_full_grouped[df_full_grouped['tackle|max']+df_full_grouped['assist|max']+df_full_grouped['pff_missedTackle|max']>=1]


In [ ]:
df_final['event']=np.where(((df_final['tackle_assist_success']==1)& (df_final['assist|max']==0)), 1, np.where((df_final['tackle_assist_success']==1)& (df_final['assist|max']==1), 2, 0))

In [ ]:
df_final['tackle_assist_success'].value_counts()

In [ ]:
df_final['event'].value_counts()

In [ ]:
df_final=pd.merge(df_final, df_speed_subset, left_on=['gameplayid', 'displayName'], right_on=['gameplayid', 'displayName'], how='left')

## Creating new features
We wanted to calculate the range for a few values, including speed, acceleration, orientation, x, and y. 

In [ ]:
def create_diff_range(df_final):
    df_final['difference_speed']=df_final['s_tackler|max']-df_final['s_ballcarrier|max']
    df_final['difference_a']=df_final['a_tackler|max']-df_final['a_ballcarrier|max']
    df_final['difference_orientation']=df_final['o_standard_tackler|max']-df_final['o_standard_ballcarrier|max']
    df_final['tackler_x_range']=df_final['x_standard_tackler|max']-df_final['x_standard_tackler|min']
    df_final['tackler_y_range']=df_final['y_standard_tackler|max']-df_final['y_standard_tackler|min']
    df_final['tackler_s_range']=df_final['s_tackler|max']-df_final['s_tackler|min']
    df_final['tackler_a_range']=df_final['a_tackler|max']-df_final['a_tackler|min']
    df_final['o_range_tackler']=df_final['o_standard_tackler|max']-df_final['o_standard_tackler|min']
    df_final['dir_range_tackler']=df_final['dir_standard_tackler|max']-df_final['dir_standard_tackler|min']
    df_final['ballcarrier_x_range']=df_final['x_standard_ballcarrier|max']-df_final['x_standard_ballcarrier|min']
    df_final['ballcarrier_y_range']=df_final['y_standard_ballcarrier|max']-df_final['y_standard_ballcarrier|min']
    df_final['ballcarrier_s_range']=df_final['s_ballcarrier|max']-df_final['s_ballcarrier|min']
    df_final['ballcarrier_a_range']=df_final['a_ballcarrier|max']-df_final['a_ballcarrier|min']
    df_final['o_range_ballcarrier']=df_final['o_standard_ballcarrier|max']-df_final['o_standard_ballcarrier|min']
    df_final['dir_range_ballcarrier']=df_final['dir_standard_ballcarrier|max']-df_final['dir_standard_ballcarrier|min']

    return df_final
df_final=create_diff_range(df_final)

In [ ]:
df_final.columns.values

### Calculating COR
Now we will compute the COR for each tackle in the tracking data bsaed on the equation noted above. 

In [ ]:
df_final['next_speed_diff']= (df_final['next_speed_tackler']-df_final['next_speed_ballcarrier'])
df_final['prior_speed_diff']=(df_final['prior_speed_tackler']-df_final['prior_speed_ballcarrier'])
df_final['COR']=np.where(df_final['tackle_assist_success']==1, (df_final['next_speed_diff']+1).astype(float)/(df_final['prior_speed_diff']+1).astype(float),0 )


In [ ]:

df_final['COR'].replace([float('inf'), float('-inf')], 0, inplace=True)
df_final['COR'].describe()

In [ ]:
df_successful_tackles=df_final[df_final['tackle_assist_success']==1]
df_successful_tackles['COR'].describe()

In [ ]:
df_final_subset= df_final.select_dtypes(include='number')
from scipy.stats import pointbiserialr
positive_variables=[]
negative_variables=[]
for col in df_final_subset.columns:
    if df_final_subset[col].isna().sum()==0:
        corr, p =pointbiserialr(df_final_subset[col], df_final_subset['tackle_assist_success'])
        if corr >=.1 and p<.05:
            positive_variables.append(col)
        if corr <=-.1  and p<.05:
            negative_variables.append(col)
        

print(positive_variables)
print(negative_variables)

In [ ]:
df_final_subset= df_final.select_dtypes(include='number')
from scipy.stats import pointbiserialr
positive_variables2=[]
negative_variables2=[]
for col in df_final_subset.columns:
    if df_final_subset[col].isna().sum()==0:
        corr, p =pointbiserialr(df_final_subset[col], df_final_subset['pff_missedTackle|max'])
        if corr >=.08 and p<.05:
            positive_variables2.append(col)
        if corr <=-.08  and p<.05:
            negative_variables2.append(col)
        

print(positive_variables2)
print(negative_variables2)

In [ ]:
df_final.fillna(0, inplace=True)

In [ ]:
positive_variables+negative_variables

In [ ]:
final_cols=[
 'o_standard_tackler|min',
 'o_standard_tackler|mean',
 'dir_standard_tackler|min',
 'a_tackler|max',
 's_ballcarrier|max',
 's_ballcarrier|mean',
 'dis_ballcarrier|max',
 'dis_ballcarrier|mean',
 'prior_speed_tackler',
 'prior_speed_ballcarrier',
 'tackler_s_range',
 'tackler_a_range',
 'o_range_tackler',
 'dir_range_tackler',
 'ballcarrier_s_range'
]

In [ ]:
# Outlier Analysis

iqr_factor = [2]
list1, list2 = [], []

for factor in iqr_factor:
    count = 0
    print(f'Outliers for {factor} IQR :')
    print('-------------------------------------')
    for col in final_cols:
    
        IQR = df_final[col].quantile(0.75) - df_final[col].quantile(0.25)
        lower_lim = df_final[col].quantile(0.25) - factor*IQR
        upper_lim = df_final[col].quantile(0.75) + factor*IQR
    
        cond = df_final[(df_final[col] < lower_lim) | (df_final[col] > upper_lim)].shape[0]
        
        if cond > 0 and factor == 1.5:
            list1.append(df_final[(df_final[col] < lower_lim) | (df_final[col] > upper_lim)].index.tolist())
        elif cond > 0 and factor == 3:
            list2.append(X[(df_final[col] < lower_lim) | (df_final[col] > upper_lim)].index.tolist())
        
        if cond > 0: print(f'{col:<30} : ', cond); count += cond
    print(f'\nTOTAL OUTLIERS FOR {factor} IQR : {count}')
    print('')

In [ ]:
def drop_outliers(df_final, col):
    IQR = df_final[col].quantile(0.75) - df_final[col].quantile(0.25)
    lower_lim = df_final[col].quantile(0.25) - (2*IQR)
    upper_lim = df_final[col].quantile(0.75) + (2*IQR)
    df_final.drop(df_final[df_final[col]>(upper_lim)].index, inplace=True)
    df_final.drop(df_final[df_final[col]<(lower_lim)].index, inplace=True)

for col in final_cols:
    drop_outliers(df_final,col)

In [ ]:
df_final.describe()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,15))
plt.title('Correlation Heatmap of Features', size=15)

sns.heatmap(df_final_subset[final_cols].astype(float).corr(),linewidths=0.1,vmax=1.0, square=True, linecolor='white', annot=True)

In [ ]:
df_set=df_final[['displayName', 'tackle|max','assist|max','pff_missedTackle|max',
                      'o_standard_tackler|min',
 'o_standard_tackler|mean',
 'dir_standard_tackler|min',
 'a_tackler|max',
 's_ballcarrier|max',
 's_ballcarrier|mean',
 'dis_ballcarrier|max',
 'dis_ballcarrier|mean',
 'prior_speed_tackler',
 'prior_speed_ballcarrier',
 'tackler_s_range',
 'tackler_a_range',
 'o_range_tackler',
 'dir_range_tackler',
 'ballcarrier_s_range'
]].groupby(by=['displayName']).mean().reset_index()
df_set

df_set_gameplays=df_final[['displayName', 'gameplayid']].groupby(by='displayName').nunique().reset_index()
df_set_tackleassist=df_final[['displayName', 'tackle_assist_success', 'pff_missedTackle|max', 'assist|max', 'tackle|max']].groupby(by='displayName').sum().reset_index()
df_set['gameplayid|nunique']=df_set_gameplays['gameplayid']
df_set[['tackle_assist_count', 'missedTacklecount', 'assist_count', 'tackle_count']]=df_set_tackleassist[['tackle_assist_success', 'pff_missedTackle|max','assist|max', 'tackle|max']]


## My Key Takeaways

I looked at the relationship between the different features and tackle success, missed tackles, and forced fumbles. Based on some of the correlations, I decided to hold on to the following features to start building a predictive model:


### Comparing Algorithms
We'll start by testing 3 classification models: Linear Discriminant Analysis, K Neighbors Classifier, and LGBM Classifier.
1. LDA finds a linear combination of features to separate classes of data. It is used for both dimensionality reduction and classification problems.
2. KNeighbors is an algorithm that takes more computing power, but it is easy to implement and adapts to new data as it comes in.
3. LGBM is our preferred model for this project because it is highly efficient, it can work with large or small datasets, and incoroporates the concept of parallel learning. 

All three models have their advantages and disadvantages, but we will start by comparing the untuned models to see which should be pursued further.

In [ ]:
# Compare Algorithms
!pip install xgboost
import pandas
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold,StratifiedGroupKFold, train_test_split
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
weak_learner = DecisionTreeClassifier(max_leaf_nodes=8)
df_final2=df_final[df_final['event']!='Assist']
X=df_final[final_cols
    
]
X.fillna(0, inplace=True) ##fill in missing pass length to 0
groups=df_final['displayName']
y=df_final['event']

# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []

models.append(('LDA', LinearDiscriminantAnalysis()))
##models.append(('KNN', KNeighborsClassifier()))
models.append(('LGBM', LGBMClassifier()))
models.append(('XBG', XGBClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('adaboost_clf', AdaBoostClassifier(
    estimator=weak_learner,
    algorithm="SAMME",
    random_state=42,
)))
models.append(('LG', LogisticRegression()))

# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'

for name, model in models:
    kfold = StratifiedKFold(n_splits=5)
    
    cv_results = model_selection.cross_val_score(model, X, y,  cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Comparing Base Models')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
ax.set_ylabel('Accuracy Score')
plt.show()

The boxplot above compares the accuracy of the three models. It is clear that LGBM has the highest accuracy and therefore we will continue to tune this model.

In [ ]:
!pip install optuna
import optuna  # pip install optuna
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import StratifiedKFold
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, stratify=y)

def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_class": 3,
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "learning_rate": trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
    }
    gbm = LGBMClassifier(**param)
    gbm.fit(X_train, y_train)
    preds = gbm.predict(X_test)
    prob = gbm.predict_proba(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

In [ ]:
"""sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100)"""


## LGBM best params


In [ ]:
#print('Best parameters:', study.best_params)

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X, y, stratify=y)

def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'mlogloss',
        'use_label_encoder': False
    }
    gbm =XGBClassifier(**param)
    gbm.fit(X_train, y_train)
    preds = gbm.predict(X_test)
    prob = gbm.predict_proba(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

#study = optuna.create_study(study_name="xgb", direction="maximize")
#study.optimize(objective, n_trials=100)

## XGBoost Best Params

In [ ]:
#print('Best parameters:', study.best_params)

In [ ]:
def objective(trial):
    """
    Objective function to be minimized.
    """
    param = {
    'n_estimators': trial.suggest_int("n_estimators", 10, 200, log=True),
    'max_depth' : trial.suggest_int("max_depth", 2, 32),
    'min_samples_split' : trial.suggest_int("min_samples_split", 2, 10),
    'min_samples_leaf' : trial.suggest_int("min_samples_leaf", 1, 10)
    }
    rf =RandomForestClassifier(**param)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)
    prob = rf.predict_proba(X_test)
    accuracy = accuracy_score(y_test, preds)
    return accuracy

#study = optuna.create_study(study_name="rf", direction="maximize")
#study.optimize(objective, n_trials=100)

## RF Best Params

In [ ]:
#print('Best parameters:', study.best_params)

In [ ]:
lgbmparams={'lambda_l1': 0.0020794164244632516, 'lambda_l2': 1.3491200862385018e-07, 'num_leaves': 183, 'feature_fraction': 0.5287512060217296, 'bagging_fraction': 0.5201519033007886, 'bagging_freq': 5, 'min_child_samples': 92, 'learning_rate': 0.037798866980162195}
xgbparams= {'max_depth': 9, 'learning_rate': 0.01572907926807038, 'n_estimators': 356, 'min_child_weight': 5, 'gamma': 0.49648197867034116, 'subsample': 0.669174163799439, 'colsample_bytree': 0.8676241143729797, 'reg_alpha': 0.00027994040114114353, 'reg_lambda': 1.8113425298795554e-07}
rfparams={'n_estimators': 187, 'max_depth': 18, 'min_samples_split': 3, 'min_samples_leaf': 2}

# 6. Modeling  <a class="anchor"  id="6"></a>

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import mean_squared_error, f1_score, precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

from sklearn.metrics import roc_auc_score

def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(actual_class)
  roc_auc_dict = {}
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_actual_class = [0 if x in other_class else 1 for x in actual_class]
    new_pred_class = [0 if x in other_class else 1 for x in pred_class]

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
    roc_auc_dict[per_class] = roc_auc

  return roc_auc_dict


dummy_clf = DummyClassifier(strategy="most_frequent")
X=df_final[final_cols
    
]
X.fillna(0, inplace=True) ##fill in missing pass length to 0
groups=df_final['displayName']
y=df_final['event']

X_train, X_test, y_train, y_test = train_test_split(X ,y, stratify=y, test_size=0.25, random_state=0)

dummy_clf.fit(X_train, y_train)



y_pred =  dummy_clf.predict(X_test)
y_proba=dummy_clf.predict_proba(X_test)
matrix = confusion_matrix(y_test, y_pred)
print("Accuracy:", accuracy_score(y_test, y_pred))

print("F1:", f1_score(y_test, y_pred,average='macro'))
print("Precision:", precision_score(y_test, y_pred,average='macro'))
print("Recall:", recall_score(y_test, y_pred,average='macro'))
print("Kappa:", cohen_kappa_score(y_test, y_pred))

print("AUC: ", roc_auc_score_multiclass(y_test, y_pred))


sns.heatmap(matrix, annot=True,  cmap="Blues", fmt="g")
plt.xlabel('Predicted'); plt.ylabel('Actual'); plt.title('Confusion Matrix')

After utilizing libraries for hyperparameter tuning, we arrived at the best paramaters for our LGBM classifier, which is implemented below. 

The next part of the notebook focuses on predicting if a tackle/assist was successful based on the conditions of the play, the tackler, and the player being tackled.

## Predicting a Successful Tackle

We want to ensure that our model isn't overfitting to the training data, so we will implement KFold cross validation to split the data into multiple train-test splits and assess the model's performance each time.

We will repeat this process and plot a confusion matrix, which shows the true positives (successful tackles), true negatives (actual missed tackles), false positives (missed tackles that we predict are successful), and false negatives (successful tackles that we predict are misses).

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, f1_score, precision_score, recall_score
from sklearn.metrics import accuracy_score
from typing import Tuple
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.under_sampling import TomekLinks

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler, NearMiss
from imblearn.under_sampling import OneSidedSelection
from imblearn.under_sampling import NeighbourhoodCleaningRule

model =LGBMClassifier(**lgbmparams)
X=df_final[final_cols
    
]
X.fillna(0, inplace=True) ##fill in missing pass length to 0
groups=df_final['displayName']
y=df_final['event']

def cross_val_predict(model, kfold : KFold, X : np.array, y : np.array) -> Tuple[np.array, np.array]:

    kfold=StratifiedKFold(n_splits=5,  random_state=None, shuffle=True)
    groups=df_final['displayName']
    
    
    
    no_classes = len(np.unique(y))
    
    actual_classes = np.empty([0], dtype=int)
    predicted_classes = np.empty([0], dtype=int)
    predicted_proba = np.empty([0, no_classes]) 

    for train_ndx, test_ndx in kfold.split(X, y):

        X_train, y_train, X_test, y_test = X[train_ndx], y[train_ndx], X[test_ndx], y[test_ndx]
       
        
        model.fit(
            X_train, y_train
        )


        


      
        actual_classes = np.append(actual_classes, y_test)
        

        predicted_classes = np.append(predicted_classes, model.predict(X_test))

        try:
            predicted_proba = np.append(predicted_proba, model.predict_proba(X_test), axis=0)
        except:
            predicted_proba = np.append(predicted_proba, np.zeros((len(X_test), no_classes), dtype=float), axis=0)

    return actual_classes, predicted_classes, predicted_proba
def plot_confusion_matrix(actual_classes : np.array, predicted_classes : np.array, sorted_labels : list):

    matrix = confusion_matrix(actual_classes, predicted_classes, labels=sorted_labels)
    print("Accuracy:", accuracy_score(actual_classes, predicted_classes))

    print("F1:", f1_score(actual_classes, predicted_classes,average='macro'))
    print("Precision:", precision_score(actual_classes, predicted_classes,average='macro'))
    print("Recall:", recall_score(actual_classes, predicted_classes,average='macro'))
    print("Kappa:" , cohen_kappa_score(actual_classes, predicted_classes))
    print("AUC: ",  roc_auc_score_multiclass(actual_classes, predicted_classes))
    plt.figure()
    sns.heatmap(matrix, annot=True, xticklabels=sorted_labels, yticklabels=sorted_labels, cmap="Blues", fmt="g")
    plt.xlabel('Predicted'); plt.ylabel('Actual'); plt.title('Confusion Matrix')

    plt.show()
    
actual_classes, predicted_classes, _ = cross_val_predict(model, kfold, X.to_numpy(), y.to_numpy())
plot_confusion_matrix(actual_classes, predicted_classes, [0,1, 2])

In [ ]:
model=XGBClassifier(**xgbparams)
actual_classes, predicted_classes, _ = cross_val_predict(model, kfold, X.to_numpy(), y.to_numpy())
plot_confusion_matrix(actual_classes, predicted_classes, [0,1, 2])

In [ ]:
model=RandomForestClassifier(**rfparams)
actual_classes, predicted_classes, _ = cross_val_predict(model, kfold, X.to_numpy(), y.to_numpy())
plot_confusion_matrix(actual_classes, predicted_classes, [0,1, 2])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X ,y, stratify=y, test_size=0.25, random_state=0)

model1=XGBClassifier(**xgbparams)
model1.fit(X_train, y_train)
probs1=model1.predict_proba(X)
model2=LGBMClassifier(**lgbmparams)
model2.fit(X_train, y_train)
probs2=model2.predict_proba(X)
model3=RandomForestClassifier(**rfparams)
model3.fit(X_train, y_train)
probs3=model3.predict_proba(X)
df_final['prob_tackle1']=probs1[:, 1]
df_final['prob_miss1']=probs1[:, 0]
df_final['prob_assist1']=probs1[:, 2]

df_final['prob_tackle2']=probs2[:, 1]
df_final['prob_miss2']=probs2[:, 0]
df_final['prob_assist2']=probs2[:, 2]


df_final['prob_tackle3']=probs3[:, 1]
df_final['prob_miss3']=probs3[:, 0]
df_final['prob_assist3']=probs3[:, 2]

df_final['prob_tackle']=(df_final['prob_tackle1']+df_final['prob_tackle2']+df_final['prob_tackle3'])/3
df_final['prob_miss']=(df_final['prob_miss1']+df_final['prob_miss2']+df_final['prob_miss3'])/3
df_final['prob_assist']=(df_final['prob_assist1']+df_final['prob_assist2']+df_final['prob_assist3'])/3
df_final['predicted_outcome']=df_final[['prob_tackle', 'prob_miss', 'prob_assist']].values.max(1)
df_final['prediction']=np.where(df_final['predicted_outcome']==df_final['prob_tackle'], 1, np.where(df_final['predicted_outcome']==df_final['prob_assist'], 2), 0)

df_final[['tackle_assist_success', 'prediction',  'prob_tackle', 'prob_miss', 'prob_assist',
                                                          'displayName', 
    'o_range_tackler', 'prior_speed_tackler', 'tacklerWeight|mean',
       'dis_ballcarrier|mean', 'dis_ballcarrier|max',
       'prior_speed_ballcarrier', 'o_standard_tackler|mean',
       'o_standard_ballcarrier|min', 'dir_range_ballcarrier',
       'relative_direction|mean',
 'force_tackler|mean', 'relative_o|mean']]

# 7. Explaining the Model  <a class="anchor"  id="7"></a>

Now that we have a model that predicts whether a tackle attempt will be successful with approximately 83% accuracy, we will work to explain the model with shap. 

## What influences Raw Tackle Ability?
The features displayed below were entered into our predictive model and the 'gain' score tells you how influential the feature is in the model compared to the other features. These are the top 10 features:


In [ ]:


gain_importance = model.feature_importances_

# Display feature importance with feature names
feature_names =  final_cols
 

gain_importance_df = pd.DataFrame({'Feature': feature_names, 'Gain': gain_importance})
gain_importance_df.sort_values(by='Gain', ascending=False).head(10)


# 8. Stats by Player <a class="anchor"  id="8"></a>

** Notes ** 
1. Raw tackling ability
2. Defensive scheme - scheme alignment, tackle probability (likely relative to offense alignment)
3. In play tackle likelihood by player 
- Data points to consider for 2 above:
    - offensive first formation
    - offensive formation at time of snap
    - defensive first formation
    - defensive formation at time of snap
- Data points to consider for 1 above:
    - stats by play by player (ex. x-range by player by play by position)
    - position 
    - raw athletic score vs. actual on field production
    - play speed for tackler
    - vision score
    - time to correct movement after ball carrier established
    - angle of approach

Now we are going to group the raw data by tackler and use the prior model to generate the probability of a player's tackle success. We will generate two unique metrics "probability of tackle" which is the natural tackling ability, and the "tackle_index" which weights players more heavily if they have played in more games.

In [ ]:
len(df_set)

In [ ]:
X=df_set[final_cols]


probs1=model1.predict_proba(X)
probs2=model2.predict_proba(X)
probs3=model3.predict_proba(X)
df_set['prob_tackle1']=probs1[:, 1]
df_set['prob_miss1']=probs1[:, 0]
df_set['prob_assist1']=probs1[:, 2]

df_set['prob_tackle2']=probs2[:, 1]
df_set['prob_miss2']=probs2[:, 0]
df_set['prob_assist2']=probs2[:, 2]


df_set['prob_tackle3']=probs3[:, 1]
df_set['prob_miss3']=probs3[:, 0]
df_set['prob_assist3']=probs3[:, 2]

df_set['RTA_tackle']=(df_final['prob_tackle1']+df_final['prob_tackle2']+df_final['prob_tackle3'])/3
df_set['RTA_miss']=(df_final['prob_miss1']+df_final['prob_miss2']+df_final['prob_miss3'])/3
df_set['RTA_assist']=(df_final['prob_assist1']+df_final['prob_assist2']+df_final['prob_assist3'])/3



df_set['RTA']=(df_set['RTA_tackle'])+(df_set['RTA_assist'])


In [ ]:
df_set['tackle_index']=round((df_set['RTA_tackle']*(df_set['tackle_count']*1)*(df_set['assist_count']*.5))-((1-df_set['RTA'])*(df_set['missedTacklecount']+1)), 2)

In [ ]:

df_tackler_merge=pd.merge(df_players, df_set[['displayName','gameplayid|nunique', 'tackle_assist_count','missedTacklecount', 'tackle_index', 'RTA']], on='displayName')
df_tackler_merge.sort_values(by='tackle_index', ascending=False, inplace=True)
df_tackler_merge.reset_index(inplace=True)
df_tackler_merge.head(30)

In [ ]:
df_tackler_merge[df_tackler_merge['gameplayid|nunique']>1].sort_values(by='RTA', ascending=False).head(20)

In [ ]:
#!pip install eli5
import eli5 as eli
tackler_name='Rashaan Evans'
index_id=df_tackler_merge[df_tackler_merge['displayName']==tackler_name].index

eli.show_prediction(model, X.iloc[index_id], feature_names = list(X.columns),  show_feature_values=True, target_names=['Missed Tackle', 'Tackle', 'Assist']) 


In [ ]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap.summary_plot(
            shap_values, X, class_names=['Missed Tackle', 'Tackle', 'Assist']
 )



# 9. NFL Use Cases <a class="anchor"  id="9"></a>

# How is this useful and innovative?
We anticipate several use cases for our predictive model:
1. **Drafting new players** - based on tracking stats from prior tackle attempts, coaches and determine if a new player's RTA makes them a good candidate for their team
2. **Adding to the PFF Player Grades** - We recommend adding a Tackle Grade in the PFF Player Grade cards. 
3. **Visualizations for interpretability** - Each player can have a visual like the one below that shows what aspects of their behavior went into their RTA and Tackle Index. In the example below, Rashaan Evans average orientation of 223 increases his RTA, while his orientation range decreases his RTA. 
4. **Monitoring Player Progress** - If a player's RTA starts to drop, it can mean that they are playing more inconsistently in terms of their tackling outcomes

## RTA = .83 | Tackle Index = 67.92

## Tackle Index Calculation
The tackle index is relative to other players. It can be negative, zero , or positive depending on the previous tackles and missed tackles that the player has made. On the other hand, RTA always ranges from 0 to 1.

# ${RTA \times Tackle Assist Count} -{[{(1-RTA) \times Missed Tackle Count}]} $

# 10. Animations/Gif <a class="anchor"  id="10"></a>

Base function courtesy of: https://www.kaggle.com/code/colinlagator/play-animation-create-gifs-in-python

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import copy

# Color are courtesy of this kaggle post:
# https://www.kaggle.com/code/huntingdata11/animated-and-interactive-nfl-plays-in-plotly
colors = {
    'ARI':"#97233F", 
    'ATL':"#A71930", 
    'BAL':'#241773', 
    'BUF':"#00338D", 
    'CAR':"#0085CA", 
    'CHI':"#C83803", 
    'CIN':"#FB4F14", 
    'CLE':"#311D00", 
    'DAL':'#003594',
    'DEN':"#FB4F14", 
    'DET':"#0076B6", 
    'GB':"#203731", 
    'HOU':"#03202F", 
    'IND':"#002C5F", 
    'JAX':"#9F792C", 
    'KC':"#E31837", 
    'LA':"#003594", 
    'LAC':"#007FC8", 
    'LV':"#000000",
    'MIA':"#008E97", 
    'MIN':"#4F2683", 
    'NE':"#002244", 
    'NO':"#D3BC8D", 
    'NYG':"#0B2265", 
    'NYJ':"#125740", 
    'PHI':"#004C54", 
    'PIT':"#FFB612", 
    'SEA':"#69BE28", 
    'SF':"#AA0000",
    'TB':'#D50A0A', 
    'TEN':"#4B92DB", 
    'WAS':"#5A1414", 
    'football':'#CBB67C'
}

def get_blank_field():
    yardlines = np.arange(100, 1100+1, 100)
    yardline_width = 4
    
    yard_mark = list(np.arange(0, 50, 10)) + [50] + list(reversed(list(np.arange(0, 50, 10))))
    font_size=40

    # Draw a green rectangle
    field = Image.new("RGB", (1200, 533), "green")
    draw = ImageDraw.Draw(field)
    
    # Draw the yardlines and the yard marker text
    assert yardline_width % 2 == 0
    for yl, ym in zip(yardlines, yard_mark):
        yl_x = (yl - (yardline_width / 2))
        draw.line([(yl_x, 0), (yl_x, 533)], width = yardline_width, fill="white")
        
        
        draw.text((yl-(font_size/2), 533-(font_size+5)), str(ym),  fill = "black")
    
    # Flip the image so the text is right side up
    field = field.transpose(1)

    return field
def draw_play_frame(frame, highlight_ids = []):

    field = get_blank_field()
    draw = ImageDraw.Draw(field)

    p_rad = 6
    padding=2
    fb_w=8
    fb_h=5
    
    df = copy.deepcopy(frame)
    
    # Round the player locations to work in the image coordinates
    plot_x = df.loc[:, "x"].apply(lambda x: round(x, 1) * 10)
    df.loc[:, "plot_x"] = plot_x
    plot_y = df.loc[:, "y"].apply(lambda x: round(x, 1) * 10)
    df.loc[:, "plot_y"] = plot_y
        
    for row in df.iterrows():
        
        x = row[1]["plot_x"]
        y = row[1]["plot_y"]
        
        # Draw a white circle behind any player dots to be highlighted
        if row[1]["nflId"] in highlight_ids:
            draw.ellipse(((x-p_rad)-padding, (y-p_rad)-padding, (x+p_rad)+padding, (y+p_rad)+padding), fill="white")

        # Draw the football
        if row[1]["club"] == "football":
            draw.ellipse((x-fb_w, y-fb_h, x+fb_w, y+fb_h), fill=colors[row[1]["club"]])
        # Draw the players with color according to the colors dictionary
        else:
            draw.ellipse((x-p_rad, y-p_rad, x+p_rad, y+p_rad), fill=colors[row[1]["club"]])
        
    return field

def finalize(field, min_x = None, max_x = None):
    """
    Finalizes the image. Does the following
    - Flips the image along the x axis
    - Optionally crops out empty field according to min_x, max_x
    """
    field = field.transpose(1)
    if (min_x is not None) & (max_x is not None):
        field = field.crop((min_x, 0, max_x, 533))

    return field
def create_play_gif(play_player_tracking_df, gif_name, crop=False, highlight_ids=[]):
    """
    Draws the play frame by frame and saves to gif
    
    Parameters
    play_player_tracking_df - A df of player_tracking_data that contains 
    a unique gameId and a unique playId
    gif_name - The name of the gif, minus the .gif extension. This is 
    added automatically.
    crop - Whether or not to crop the gif to only contain the minimum and
    maximum x values within the entire play
    highlight_ids - The ids of players to draw a white circle behind them in
    order to call attention to them.
    
    """
    min_x = (round(play_player_tracking_df.x.min(), 1) * 10) - 50
    max_x = (round(play_player_tracking_df.x.max(), 1) * 10) + 50
    
    gif_frames = []
    frames = play_player_tracking_df["frameId"].values
    for i in frames:
        frame = play_player_tracking_df[play_player_tracking_df["frameId"] == i].copy(deep=True)
        field = get_blank_field()
        
        field = draw_play_frame(frame, highlight_ids)
            
        if crop:
            field = finalize(field, min_x = min_x, max_x =  max_x)
        else:
            field = finalize(field)
            
        gif_frames.append(field)
    frame_one = gif_frames[0]
    print(frame_one)
    frame_one.save(f"{gif_name}.gif", format="GIF", append_images=gif_frames,
                save_all=True, duration=100, loop=0)

    return gif_name
# Display the gif
def show_gif(gif_name):
    import base64
    from IPython import display
    
    with open(str(gif_name)+".gif", 'rb') as fd:
        b64 = base64.b64encode(fd.read()).decode('ascii')
    return display.HTML(f'<img src="data:image/gif;base64,{b64}" />')

In [ ]:
gameplay='20220908002712'


     
show_gif(create_play_gif(df_tracking[df_tracking['gameplayid']==gameplay], str(gameplay)+'.gif'))




# Data Dictionary
## Tackles data
- **gameId:** Game identifier, unique (numeric)
- **playId:** Play identifier, not unique across games (numeric)
- **nflId:** Player identification number, unique across players (numeric)
- **tackle:** Indicator for whether the given player made a tackle on the play (binary)
- **assist:** Indicator for whether the given player made an assist tackle on the play (binary)
- **forcedFumble:** Indicator for whether the given player forced a fumble on the play (binary)
- **pff_missedTackle:** Provided by Pro Football Focus (PFF). Indicator for whether the given player missed a tackle on the play (binary)

## Tracking data

- **gameId:** Game identifier, unique (numeric)
- **playId:** Play identifier, not unique across games (numeric)
- **nflId:** Player identification number, unique across players. When value is NA, row corresponds to ball. (numeric)
- **displayName:** Player name (text)
- **frameId:** Frame identifier for each play, starting at 1 (numeric)
- **time:** Time stamp of play (time, yyyy-mm-dd, hh:mm:ss)
- **jerseyNumber:** Jersey number of player (numeric)
- **club:** Team abbrevation of corresponding player (text)
- **playDirection:** Direction that the offense is moving (left or right)
- **x:** Player position along the long axis of the field, 0 - 120 yards. See Figure 1 below. (numeric)
- **y:** Player position along the short axis of the field, 0 - 53.3 yards. See Figure 1 below. (numeric)
- **s:** Speed in yards/second (numeric)
- **a:** Speed in yards/second^2 (numeric)
- **dis:** Distance traveled from prior time point, in yards (numeric)
- **o:** Player orientation (deg), 0 - 360 degrees (numeric)
- **dir:** Angle of player motion (deg), 0 - 360 degrees (numeric)
- **event:** Tagged play details, including moment of ball snap, pass release, pass catch, tackle, etc (text)

# Resources Used


- https://francis-press.com/uploads/papers/ndkLZY5nQLH6nilaE0nfMi0dxkAogA90wpqUB2vK.pdf
- https://digitalcommons.unl.edu/cgi/viewcontent.cgi?article=1079&context=cbbbpapers
- https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9459770/
- https://www.frontiersin.org/articles/10.3389/fspor.2021.669845/full
- https://entertainment.howstuffworks.com/physics-of-football.htm